In [1]:
# Librería para calcular de forma fácil la exactitud

%pip install torchmetrics --quiet

Note: you may need to restart the kernel to use updated packages.


Usando una librería de python (Keras o Pytorch) desarrolle los modelos de Convolutional Neural Networks que se describen para cada
problema en particular (usar una jupyter notebook por cada dataset y entregar todos en una carpeta zip).


1. MNIST Dataset. Escalar las imágenes en un rango de [0,1]. Usar 10% del conjunto de prueba para validación en el entrenamiento. Usar
como función de perdida categorical_crossentropy, optimizador adam y métrica exactitud durante el entrenamiento de 15 épocas. Aplicar
zero-padding en las capas conv2d. Reportar valor de perdida y exactitud para entrenamiento y prueba. Tomando como referencia el
siguiente diagrama:

1. Escalar las imágenes en un rango de [0,1]. 


In [1]:
import torchvision.datasets as datasets
from torchvision import transforms

# Normalizamos las imágenes en rango de 0 y 1
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Cargar dataset y normalizar
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

2. Usar 10% del conjunto de prueba para validación en el entrenamiento. 


In [5]:
from torch.utils.data import random_split

# Usar 10% del conjunto de prueba para validación en el entrenamiento. 
test_dataset_size = len(mnist_testset)

validation_size = int(test_dataset_size * 0.1)
test_size = test_dataset_size - validation_size

mnist_validationset, mnist_testset = random_split(mnist_testset, [validation_size, test_size])

print(f"Tamaño del conjunto de entrenamiento: {len(mnist_trainset)}")
print(f"Tamaño del conjunto de prueba: {len(mnist_testset)}")
print(f"Tamaño del conjunto de validación: {len(mnist_validationset)}")

Tamaño del conjunto de entrenamiento: 60000
Tamaño del conjunto de prueba: 9000
Tamaño del conjunto de validación: 1000


In [2]:
from torch.utils.data import DataLoader

batch_size = 64

# Crear los conjuntos de entrenamiento, prueba y validación
train_data = DataLoader(dataset=mnist_trainset, 
                        batch_size=batch_size,
                        num_workers=2,
                        shuffle=True)

test_data = DataLoader(dataset=mnist_trainset, 
                        batch_size=batch_size,
                        num_workers=2)

val_data = DataLoader(dataset=mnist_validationset, 
                        batch_size=batch_size,
                        num_workers=2)

NameError: name 'mnist_validationset' is not defined

![Red](https://neji3cswyd.clg07azjl.paperspacegradient.com/files/red.png?token=df62b5170a9372b81ef5eae580c731a5)

4. Aplicar zero-padding en las capas conv2d. Reportar valor de perdida y exactitud para entrenamiento y prueba.

a) Implemente la CNN del diagrama omitiendo las capas dropout e implemente EarlyStopping monitoreando la perdida del conjunto de
validación y una paciencia de 2.



In [9]:
import torch
import torch.nn as nn


class NeuralNetwork(nn.Module):
    def __init__(self, num_classes):
        super(NeuralNetwork, self).__init__()

        self.network = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Flatten(),
            nn.Linear(in_features=9216, out_features=128),
            nn.Linear(in_features=128, out_features=num_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.network(x)
    

model = NeuralNetwork(num_classes=10)
print(model)

NeuralNetwork(
  (network): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Flatten(start_dim=1, end_dim=-1)
    (6): Linear(in_features=9216, out_features=128, bias=True)
    (7): Linear(in_features=128, out_features=10, bias=True)
    (8): Softmax(dim=1)
  )
)


In [10]:
import torch

def get_device():
    device = ""
    if torch.cuda.is_available():
        !nvidia-smi
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    return device

device = get_device()
device

device(type='cpu')

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
import numpy as np

from pytorchtools import EarlyStopping

def train(model, patience, epochs):
    train_losses = []
    valid_losses = []